<a href="https://colab.research.google.com/github/plus2net/Python-basics/blob/main/pytorch_basic_titanic_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess the Titanic dataset
df = sns.load_dataset('titanic')[['age', 'fare', 'sex', 'pclass', 'survived']].dropna()
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# Features and labels
X = df[['age', 'fare', 'sex', 'pclass']].values
y = df['survived'].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create data loader
train_ds = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)

# Define the neural network
class TitanicNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

model = TitanicNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    model.train()
    for xb, yb in train_loader:
        preds = model(xb)
        loss = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Evaluation on test set
model.eval()
with torch.no_grad():
    preds = model(X_test)
    preds = (preds > 0.5).float()
    accuracy = (preds.eq(y_test).sum().item() / len(y_test))
    print(f'Test Accuracy: {accuracy:.4f}')
torch.save(model.state_dict(),'smo.pt') # creating a Model file and saving

Epoch 1, Loss: 0.6869
Epoch 2, Loss: 0.6801
Epoch 3, Loss: 0.6812
Epoch 4, Loss: 0.6558
Epoch 5, Loss: 0.6392
Epoch 6, Loss: 0.5929
Epoch 7, Loss: 0.5888
Epoch 8, Loss: 0.5751
Epoch 9, Loss: 0.5144
Epoch 10, Loss: 0.5590
Epoch 11, Loss: 0.5677
Epoch 12, Loss: 0.4687
Epoch 13, Loss: 0.4615
Epoch 14, Loss: 0.3335
Epoch 15, Loss: 0.4906
Epoch 16, Loss: 0.3658
Epoch 17, Loss: 0.2681
Epoch 18, Loss: 0.3153
Epoch 19, Loss: 0.3038
Epoch 20, Loss: 0.3570
Test Accuracy: 0.7343


Using the above created model ( last line )  file to load the pytorch model. Without this file also we can pass the model to below code.

In [ ]:
#model=TitanicNN()
#model.load_state_dict(torch.load('smo.pt'))

<All keys matched successfully>

In [ ]:
import numpy as np

# 1. Create a sample: [age, fare, sex, pclass]
#    e.g.  28 years old, $72 fare, female (1), 1st class (1)
raw_sample = np.array([[28, 72.0, 1, 1]])

# 2. Scale using the same StandardScaler
sample_scaled = scaler.transform(raw_sample)

# 3. Convert to tensor
sample_tensor = torch.tensor(sample_scaled, dtype=torch.float32)

# 4. Model inference
model.eval()
with torch.no_grad():
    prob = model(sample_tensor).item()           # Probability of survival
    pred = int(prob > 0.5)                       # 1 if survived, 0 otherwise

print(f"Survival probability: {prob:.4f}")
print("Prediction:", "Survived" if pred==1 else "Did not survive")


Survival probability: 0.9741
Prediction: Survived
